In [ ]:
#librerie
import os
from PIL import Image
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Funzione per leggere le immagini e creare una lista di etichette
def carica_immagini_e_etichette(cartella_principale):
    etichette = []
    immagini = []

    # Scorri le sottocartelle nella cartella principale
    for cartella in os.listdir(cartella_principale):
        cartella_path = os.path.join(cartella_principale, cartella)

        # Verifica che sia una cartella
        if os.path.isdir(cartella_path):
            # Scorri le immagini all'interno della cartella
            for file in os.listdir(cartella_path):
                file_path = os.path.join(cartella_path, file)

                # Verifica che il file sia un'immagine (controlla l'estensione)
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    try:
                        # Apre l'immagine per verificarne la validità
                        img = Image.open(file_path)
                        img.verify()  # Verifica se l'immagine è valida

                        # Aggiungi l'immagine e la sua etichetta (nome della cartella)
                        immagini.append(file_path)
                        etichette.append(cartella)
                    except (IOError, SyntaxError):
                        # Se l'immagine non è valida, la ignora
                        continue

    return immagini, etichette


build a visual vocabulary:


*   sample many (10K to 100K) SIFT descriptors from the images of
the training set (you either use a detector or sample on a grid in the
scale-space);
*  cluster them using k-means (the choice of the number of clusters is
up to you, and you should experiment with different values, but you
could start with a few dozens);
*  collect (and save for future use) the clusters’ centroids which repre-
sent the k 128-dimensional visual words.

In [ ]:
#carico immagini training
cartella_training = '/home/eva/Scrivania/esame_cv/train'
images_training, train_labels  = carica_immagini_e_etichette(cartella_training) #metti la path cartella

In [ ]:
#SIFT descriptor
all_descriptors = []
k_p = 10000 #prova
for image_path in images_training:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create(k_p)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is not None:
        all_descriptors.append(descriptors)
all_descriptors = np.vstack(all_descriptors)

In [ ]:
#Cluster descriptors per visual words (vocabulary)
k = 60 #tuning necessario
kmeans = KMeans(n_clusters=k)
kmeans.fit(all_descriptors)
visual_words = kmeans.cluster_centers_

Represent each image of the training set as a normalized histogram having
k bins, each corresponding to a visual word; a possibility is to perform a
rather dense sampling in space and scale; another possibility is to use the
3SIFT detector to find the points in scale-space where the descriptor is
computed. In any case, each computed descriptor will increase the value
of the bin corresponding to the closest visual word.

In [ ]:
#da ottimizzare il codice
train_histograms = []
for image_path in images_training:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create(k_p) # non so se sia necessario ricalcolare il sift e se serva lo stesso numero di punti
    keypoints, descriptors = sift.detectAndCompute(image, None)

    hist = np.zeros(k)
    if descriptors is not None:
        for descriptor in descriptors:
            distances = np.linalg.norm(visual_words - descriptor, axis=1)
            visual_word_index = np.argmin(distances)
            hist[visual_word_index] += 1

    hist = hist / np.sum(hist)  #normalizzo l'istogramma
    train_histograms.append(hist)

Employ a nearest neighbor classifier and evaluate its performance:
• compute the normalized histogram for the test image to be classified;
• assign to the image the class corresponding to the training image
having the closest histogram.
• repeat for all the test images and build a confusion matrix.

In [ ]:
#Carico le immagini di test
cartella_test = '/home/eva/Scrivania/esame_cv/test'
images_test, test_labels  = carica_immagini_e_etichette(cartella_test)

In [ ]:
#istogrammi test ---- nota: ho scritto la versione lunga del codice, meglio definire funzioni apposite, forse ce n'è una in opncv
test_histograms = []
for image_path in images_test:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create(k_p) # al solito uso sempre lo stesso numero di punti per adesso
    keypoints, descriptors = sift.detectAndCompute(image, None)

    hist = np.zeros(k)
    if descriptors is not None:
        for descriptor in descriptors:
            distances = np.linalg.norm(visual_words - descriptor, axis=1)
            visual_word_index = np.argmin(distances)
            hist[visual_word_index] += 1

    hist = hist / np.sum(hist)  #normalizzo l'istogramma
    test_histograms.append(hist)

In [ ]:
#knn non è efficiente si deve cambiare algoritmo poi vediamo cosa usare
knn = NearestNeighbors(n_neighbors=24) #ho usato un numero a caso
knn.fit(train_histograms)

# Find the nearest neighbors and classify based on the closest training image
predictions = knn.kneighbors(test_histograms, return_distance=False)
predicted_labels = [train_labels[i[0]] for i in predictions]

In [ ]:
#confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)
print("Confusion Matrix:")
print(cm) #dopo ci metto il grafico carino

Train a multiclass linear Support Vector Machine, using the one-vs-rest
approach (you will need to train 15 binary classifiers having the normalized
histograms as the input vectors and positive labels for the “one” class and
negative for the “rest.”)

In [ ]:
svm = SVC(kernel='linear', random_state=42)
ovr_classifier = OneVsRestClassifier(svm)
x = np.vstack(train_histograms)
y = np.array(train_labels)
ovr_classifier.fit(x, train_labels)

5. Evaluate the multiclass SVM:
• compute the normalized histogram for the test image to be classified;
• compute the real-valued output of each of the SVMs, using that his-
togram as input;
• assign to the image the class corresponding to the SVM having the
greatest real-valued output.
• repeat for all the test images and build a confusion matrix.

In [ ]:
#devo riguardare meglio questo punto
test = np.vstack(test_histograms)
y_pred = ovr_classifier.predict(test)
y_test = np.array(test_labels)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
#confusion matrix ---- finisco dopo

EXRA
6. svm cambio kernel

In [ ]:
#dovrebbe essere una cosa simile ma devo provarla meglio
from sklearn.metrics.pairwise import pairwise_kernels

#gaussiana generalizzata
def chi_squared_kernel(X, Y=None, gamma=1.0):
    if Y is None:
        Y = X

    #chi^2
    diff = X[:, np.newaxis, :] - Y[np.newaxis, :, :]
    sum_X = X[:, np.newaxis, :] + Y[np.newaxis, :, :]
    chi_squared = np.sum((diff**2) / (sum_X + 1e-10), axis=2)

    # Generalized Gaussian kernel, using the chi-squared distance
    return np.exp(-gamma * chi_squared)

svm = SVC(kernel='precomputed')